# Open AI மாதிரிகளை நுட்பமாகச் சரிசெய்தல்

இந்த நோட்புக் Open AI இன் [நுட்பமாகச் சரிசெய்தல்](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) ஆவணத்தில் வழங்கப்பட்ட தற்போதைய வழிகாட்டுதலின் அடிப்படையில் உருவாக்கப்பட்டுள்ளது.

நுட்பமாகச் சரிசெய்தல் உங்கள் பயன்பாட்டிற்கான அடித்தள மாதிரிகளின் செயல்திறனை மேம்படுத்துகிறது, அதற்கான குறிப்பிட்ட பயன்பாடு அல்லது சூழ்நிலைக்கு தொடர்புடைய கூடுதல் தரவு மற்றும் சூழலை கொண்டு மீண்டும் பயிற்சி அளிப்பதன் மூலம். _சிறிய எடுத்துக்காட்டு கற்றல்_ மற்றும் _திரும்ப பெறுதல் மேம்படுத்தப்பட்ட உருவாக்கம்_ போன்ற ப்ராம்ட் பொறியியல் தொழில்நுட்பங்கள் தரத்தை மேம்படுத்த தொடர்புடைய தரவுடன் இயல்புநிலை ப்ராம்ட்டை மேம்படுத்த அனுமதிக்கின்றன. இருப்பினும், இந்த அணுகுமுறைகள் இலக்காகக் கொண்ட அடித்தள மாதிரியின் அதிகபட்ச டோக்கன் சாளர அளவால் வரம்பு விதிக்கப்படுகின்றன.

நுட்பமாகச் சரிசெய்தலுடன், தேவையான தரவுடன் மாதிரியை தானாகவே மீண்டும் பயிற்சி அளிக்கிறோம் (அதாவது அதிகமான எடுத்துக்காட்டுகளை அதிகபட்ச டோக்கன் சாளரத்தில் பொருந்தக்கூடிய அளவுக்கு விட அதிகமாக பயன்படுத்த முடியும்) - மேலும் இன்ஃபெரன்ஸ் நேரத்தில் எடுத்துக்காட்டுகளை வழங்க தேவையில்லாத _தனிப்பயன்_ மாதிரியை வெளியிடுகிறோம். இது நமது ப்ராம்ட் வடிவமைப்பின் செயல்திறனை மட்டுமல்லாமல் (டோக்கன் சாளரத்தை பிற விஷயங்களுக்கு பயன்படுத்துவதில் அதிக சுதந்திரம் கிடைக்கும்) நமது செலவுகளையும் மேம்படுத்தக்கூடும் (இன்ஃபெரன்ஸ் நேரத்தில் மாதிரிக்கு அனுப்ப வேண்டிய டோக்கன்களின் எண்ணிக்கையை குறைப்பதன் மூலம்).

நுட்பமாகச் சரிசெய்தல் 4 படிகள் கொண்டது:
1. பயிற்சி தரவை தயார் செய்து பதிவேற்றுக.
1. நுட்பமாகச் சரிசெய்த மாதிரியை பெற பயிற்சி பணியை இயக்குக.
1. நுட்பமாகச் சரிசெய்த மாதிரியை மதிப்பாய்வு செய்து தரத்திற்காக மீண்டும் முயற்சிக்கவும்.
1. திருப்தி அடைந்தபின் இன்ஃபெரன்ஸுக்காக நுட்பமாகச் சரிசெய்த மாதிரியை வெளியிடுக.

அனைத்து அடித்தள மாதிரிகளும் நுட்பமாகச் சரிசெய்தலை ஆதரிக்காது என்பதை கவனிக்கவும் - சமீபத்திய தகவலுக்கு [OpenAI ஆவணத்தைச் சரிபார்க்கவும்](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst). நீங்கள் முந்தைய நுட்பமாகச் சரிசெய்த மாதிரியையும் மீண்டும் நுட்பமாகச் சரிசெய்யலாம். இந்த பயிற்சியில், நுட்பமாகச் சரிசெய்தலுக்கான இலக்காக `gpt-35-turbo` அடித்தள மாதிரியை பயன்படுத்துவோம்.

---


### படி 1.1: உங்கள் தரவுத்தொகுப்பை தயார் செய்யவும்

ஒரு கூறு குறும்படம் மூலம் ஒரு கூறு பற்றி கேள்விகளுக்கு பதிலளிப்பதன் மூலம் கூறுகளின் காலாண்டு அட்டவணையை புரிந்துகொள்ள உதவும் ஒரு சாட்போட்டை உருவாக்குவோம். _இந்த_ எளிய பயிற்சியில், தரவின் எதிர்பார்க்கப்படும் வடிவத்தை காட்டும் சில மாதிரி பதில்களின் உதாரணங்களுடன் மாதிரியை பயிற்சி செய்ய ஒரு தரவுத்தொகுப்பை உருவாக்குவோம். ஒரு உண்மையான பயன்பாட்டில், நீங்கள் பல உதாரணங்களுடன் ஒரு தரவுத்தொகுப்பை உருவாக்க வேண்டியிருக்கும். உங்கள் பயன்பாட்டு துறைக்கு ஏற்ற ஒரு திறந்த தரவுத்தொகுப்பு இருந்தால், அதை பயன்படுத்தி நன்றாக வடிவமைத்து நுட்பமாக பயிற்சி செய்யலாம்.

நாம் `gpt-35-turbo` மீது கவனம் செலுத்தி ஒரே சுற்று பதிலை (சாட் நிறைவு) எதிர்பார்க்கும் போது, OpenAI சாட் நிறைவு தேவைகளை பிரதிபலிக்கும் [இந்த பரிந்துரைக்கப்பட்ட வடிவத்தை](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) பயன்படுத்தி உதாரணங்களை உருவாக்கலாம். நீங்கள் பல சுற்று உரையாடல் உள்ளடக்கத்தை எதிர்பார்க்கும் போது, [பல சுற்று உதாரண வடிவத்தை](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) பயன்படுத்துவீர்கள், இது fine-tuning செயல்முறையில் எந்த செய்திகளை பயன்படுத்த வேண்டும் (அல்லது பயன்படுத்தக்கூடாது) என்பதை குறிக்க `weight` அளவுருவை கொண்டுள்ளது.

நாம் இங்கே எங்கள் பயிற்சிக்காக எளிய ஒரே சுற்று வடிவத்தை பயன்படுத்துவோம். தரவு [jsonl வடிவத்தில்](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) உள்ளது, ஒவ்வொரு வரியிலும் ஒரு பதிவாகும், ஒவ்வொன்றும் JSON வடிவமைக்கப்பட்ட பொருளாக பிரதிபலிக்கப்படுகிறது. கீழே உள்ள துணுக்கில் 2 பதிவுகள் மாதிரியாக காட்டப்பட்டுள்ளன - முழு மாதிரி தொகுப்புக்காக [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) ஐப் பார்க்கவும் (10 உதாரணங்கள்) நாங்கள் நுட்ப பயிற்சிக்காக பயன்படுத்துவோம். **குறிப்பு:** ஒவ்வொரு பதிவு _ஒரே வரியில்_ வரையறுக்கப்பட வேண்டும் (வடிவமைக்கப்பட்ட JSON கோப்பில் பொதுவாகப் போல வரிகளை பிரிக்க வேண்டாம்)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

உண்மையான பயன்பாட்டில் நல்ல முடிவுகளுக்காக நீங்கள் மிகவும் பெரிய உதாரண தொகுப்பைத் தேவைப்படுத்துவீர்கள் - பதில்களின் தரம் மற்றும் நுட்ப பயிற்சிக்கான நேரம்/செலவுகளுக்கு இடையேயான சமநிலை இருக்கும். நாங்கள் ஒரு சிறிய தொகுப்பை பயன்படுத்துகிறோம், அதனால் நுட்ப பயிற்சியை விரைவாக முடித்து செயல்முறையை விளக்க முடியும். மேலும் சிக்கலான நுட்ப பயிற்சி பயிற்சிக்காக [இந்த OpenAI சமையல் புத்தகம் உதாரணத்தை](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) பார்க்கவும்.


---

### படி 1.2 உங்கள் தரவுத்தொகுப்பை பதிவேற்றவும்

கோப்புகள் API-ஐப் பயன்படுத்தி தரவை [இங்கே விவரிக்கப்பட்டபடி](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) பதிவேற்றவும். இந்தக் குறியீட்டை இயக்க, நீங்கள் முதலில் பின்வரும் படிகளை செய்திருக்க வேண்டும்:
 - `openai` Python தொகுப்பை நிறுவியிருக்க வேண்டும் (சமீபத்திய அம்சங்களுக்கு >=0.28.0 பதிப்பை பயன்படுத்துவதை உறுதி செய்யவும்)
 - `OPENAI_API_KEY` சுற்றுச்சூழல் மாறியை உங்கள் OpenAI API விசையுடன் அமைத்திருக்க வேண்டும்
மேலும் அறிய, பாடத்திட்டத்திற்கான [அமைப்பு வழிகாட்டி](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) ஐப் பார்க்கவும்.

இப்போது, உங்கள் உள்ளூர் JSONL கோப்பிலிருந்து பதிவேற்ற கோப்பை உருவாக்கக் குறியீட்டை இயக்கவும்.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### படி 2.1: SDK உடன் நுட்பமாக்கும் வேலை உருவாக்கவும்


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### படி 2.2: வேலை நிலையை சரிபார்க்கவும்

`client.fine_tuning.jobs` API உடன் நீங்கள் செய்யக்கூடிய சில விஷயங்கள் இங்கே உள்ளன:
- `client.fine_tuning.jobs.list(limit=<n>)` - கடைசியாக செய்யப்பட்ட n நுணுக்கமயமாக்கல் வேலைகளை பட்டியலிடு
- `client.fine_tuning.jobs.retrieve(<job_id>)` - ஒரு குறிப்பிட்ட நுணுக்கமயமாக்கல் வேலை விவரங்களைப் பெறுக
- `client.fine_tuning.jobs.cancel(<job_id>)` - ஒரு நுணுக்கமயமாக்கல் வேலைத்தை ரத்து செய்க
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - வேலைக்கான n நிகழ்வுகளை பட்டியலிடு
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

செயல்முறை முதல் படி _பயிற்சி கோப்பை சரிபார்த்தல்_ ஆகும், தரவு சரியான வடிவத்தில் உள்ளதா என்பதை உறுதிப்படுத்த.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### படி 2.3: முன்னேற்றத்தை கண்காணிக்க நிகழ்வுகளை பின்தொடரவும்


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### படி 2.4: OpenAI டாஷ்போர்டில் நிலையைப் பார்க்கவும்


நீங்கள் OpenAI இணையதளத்தைப் பார்வையிடுவதன் மூலம் மற்றும் தளத்தின் _Fine-tuning_ பகுதியை ஆராய்வதன் மூலம் நிலையைப் பார்க்கலாம். இது தற்போதைய பணியின் நிலையை காட்டும், மேலும் முந்தைய பணியின் செயல்பாட்டு வரலாற்றையும் கண்காணிக்க அனுமதிக்கும். இந்த ஸ்கிரீன்ஷாட்டில், முந்தைய செயல்பாடு தோல்வியடைந்தது மற்றும் இரண்டாவது ஓட்டம் வெற்றிகரமாக முடிந்தது என்பதை நீங்கள் காணலாம். சூழ்நிலைக்காக, இது முதல் ஓட்டம் தவறான வடிவமைப்பில் உள்ள JSON கோப்பைப் பயன்படுத்தியபோது நடந்தது - சரி செய்த பிறகு, இரண்டாவது ஓட்டம் வெற்றிகரமாக முடிந்தது மற்றும் மாதிரியை பயன்படுத்துவதற்கு கிடைக்கச் செய்தது.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba.ta.png)


நீங்கள் கீழே மேலும் ஸ்க்ரோல் செய்து காட்சி டாஷ்போர்டில் நிலை செய்திகள் மற்றும் அளவுகோல்களைப் பார்க்கலாம், கீழே காட்டப்பட்டுள்ளபடி:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b.ta.png) |  ![Metrics](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a65229.ta.png)|


---

### படி 3.1: ஐடி பெறவும் மற்றும் குறியீட்டில் நுட்பமாகத் திருத்தப்பட்ட மாதிரியை சோதிக்கவும்


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### படி 3.2: பிளேகிரவுண்டில் நுட்பமாக பயிற்சி பெற்ற மாதிரியை ஏற்றவும் சோதிக்கவும்

நீங்கள் இப்போது நுட்பமாக பயிற்சி பெற்ற மாதிரியை இரண்டு வழிகளில் சோதிக்கலாம். முதலில், நீங்கள் பிளேகிரவுண்டை பார்வையிட்டு, மாதிரிகள் தள்ளுபடி பட்டியலில் இருந்து உங்கள் புதிய நுட்பமாக பயிற்சி பெற்ற மாதிரியை தேர்ந்தெடுக்கலாம். மற்றொரு விருப்பம், நுட்பமாக பயிற்சி பெறுதல் குழுவில் காணப்படும் "பிளேகிரவுண்ட்" விருப்பத்தை பயன்படுத்துவதாகும் (மேலே உள்ள ஸ்கிரீன்ஷாட்டைப் பார்க்கவும்) இது அடிப்படை மற்றும் நுட்பமாக பயிற்சி பெற்ற மாதிரி பதிப்புகளை பக்கமெதிராக காட்டும் _ஒப்பீட்டு_ பார்வையை துவக்குகிறது, இது விரைவான மதிப்பீட்டிற்கு உதவுகிறது.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016.ta.png)

உங்கள் பயிற்சி தரவுகளில் பயன்படுத்திய அமைப்பு சூழலை நிரப்பி, உங்கள் சோதனை கேள்வியை வழங்கவும். இரு பக்கங்களும் ஒரே அமைப்பு சூழலும் கேள்வியும் கொண்டு புதுப்பிக்கப்படுவதை நீங்கள் கவனிப்பீர்கள். ஒப்பீட்டை இயக்கி, அவற்றின் வெளியீடுகளுக்கு இடையேயான வேறுபாட்டை காணலாம். _நுட்பமாக பயிற்சி பெற்ற மாதிரி உங்கள் எடுத்துக்காட்டுகளில் வழங்கிய வடிவத்தில் பதிலை வழங்குவதை கவனியுங்கள், அதே சமயம் அடிப்படை மாதிரி அமைப்பு உத்தரவினை பின்பற்றுகிறது_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350.ta.png)

ஒப்பீடு ஒவ்வொரு மாதிரிக்கும் டோக்கன் எண்ணிக்கைகளையும், ஊகிக்க எடுத்த நேரத்தையும் வழங்குகிறது என்பதை நீங்கள் கவனிப்பீர்கள். **இந்த குறிப்பிட்ட எடுத்துக்காட்டு ஒரு எளிய செயல்முறை விளக்கமாகும், ஆனால் உண்மையான தரவுத்தொகுப்பு அல்லது சூழலை பிரதிபலிக்கவில்லை**. இரு மாதிரிகளும் ஒரே டோக்கன் எண்ணிக்கையை (அமைப்பு சூழலும் பயனர் உத்தரவும் ஒரே மாதிரியாக இருப்பதால்) காட்டினாலும், நுட்பமாக பயிற்சி பெற்ற மாதிரி ஊகிக்க அதிக நேரம் எடுத்துக்கொள்கிறது (தனிப்பயன் மாதிரி).

உண்மையான சூழல்களில், நீங்கள் இத்தகைய விளையாட்டு எடுத்துக்காட்டைப் பயன்படுத்த மாட்டீர்கள், ஆனால் உண்மையான தரவுக்கு எதிராக நுட்பமாக பயிற்சி பெறுவீர்கள் (எ.கா., வாடிக்கையாளர் சேவைக்கான தயாரிப்பு பட்டியல்) அங்கு பதிலின் தரம் மிகவும் தெளிவாக இருக்கும். _அந்த_ சூழலில், அடிப்படை மாதிரியுடன் சமமான பதில்தரத்தை பெற, அதிக தனிப்பயன் உத்தரவமைப்பை உருவாக்க வேண்டியிருக்கும், இது டோக்கன் பயன்பாட்டையும் ஊகிக்க எடுத்த நேரத்தையும் அதிகரிக்கும். _இதனை முயற்சிக்க, OpenAI சமையல் புத்தகத்தில் உள்ள நுட்பமாக பயிற்சி பெறுதல் எடுத்துக்காட்டுகளை பார்வையிடவும்._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**குறிப்பு**:  
இந்த ஆவணம் AI மொழிபெயர்ப்பு சேவை [Co-op Translator](https://github.com/Azure/co-op-translator) மூலம் மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சித்தாலும், தானாக செய்யப்பட்ட மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறுகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனிக்கவும். அசல் ஆவணம் அதன் சொந்த மொழியில் அதிகாரப்பூர்வ மூலமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பை பரிந்துரைக்கிறோம். இந்த மொழிபெயர்ப்பின் பயன்பாட்டால் ஏற்படும் எந்தவொரு தவறான புரிதலுக்கும் அல்லது தவறான விளக்கங்களுக்கும் நாங்கள் பொறுப்பேற்கமாட்டோம்.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
